In [1]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops.clip_ops import clip_by_global_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds


# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, Flatten, Dense, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split


from deeplab2.model.pixel_encoder import moat
from AdaBelief_tf import AdaBeliefOptimizer

### https://www.tensorflow.org/datasets/catalog/cifar100

In [2]:
# 'cifar100' 대신 'food101' 이나 다른 데이터셋 이름 넣으면 ok
from tensorflow.keras.datasets import cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

train_ohe_labels = to_categorical(train_labels)
test_ohe_labels = to_categorical(test_labels)

tr_images, val_images, tr_ohe_labels, val_ohe_labels = train_test_split(train_images, train_ohe_labels,test_size=0.1,random_state=34)

print('Train:', tr_images.shape, tr_ohe_labels.shape)
print('Valid:', val_images.shape, val_ohe_labels.shape)
print('Test:', test_images.shape, test_ohe_labels.shape)

Train: (45000, 32, 32, 3) (45000, 100)
Valid: (5000, 32, 32, 3) (5000, 100)
Test: (10000, 32, 32, 3) (10000, 100)


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)

test_datagen = ImageDataGenerator(
    rescale=1./255,)

In [4]:
# Numpy Array Iterator 객체 생성하여 모델 인풋으로 배치 사이즈만큼 집어넣을 준비
train_generator = train_datagen.flow(x=tr_images, y=tr_ohe_labels, batch_size=32, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_ohe_labels, batch_size=32, shuffle=False)
test_generator = test_datagen.flow(x=test_images, y=test_ohe_labels, batch_size=32, shuffle=False)

In [5]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1 = tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model():
    return tf.keras.models.Model(moat1.input, out1)

In [ ]:
model1 = create_model2()
#model1.load_weights('pet_final_disjoint1_1.hdf5')

opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
checkpointer = ModelCheckpoint(filepath='pet_final_disjoint1_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('pet_final_disjoint1_2.log')
history = model1.fit_generator(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=80,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])